**Workflow**:

---

1. Current Recharge Trending - Break into Yearly, Quarterly, Weekly Seasonality, plus Trend
2. Optimized Forecasting model - forecast for the next 7 days
3. daily update workflow
4. Final output - Recharge forecast Vs. Actual, Additives, Error accuracy.

In [7]:
#Import Packages
import pandas as pd
from prophet import Prophet
from google.colab import files
import xlrd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose



In [25]:
#Current shape of the Dataset - Add seasonal decomposition

df = pd.read_excel('/content/Ufone.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

#Canvas

df = df.set_index(['ds'])
df = df.asfreq('D')
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] =df['y'].interpolate()
df['y'] = df['y'].ffill().bfill()
df= df.sort_index()
print(df['y'].isna().sum())
#Weekly

weekly_decompose = seasonal_decompose(df['y'], model='additive', period=7)
#weekly_decompose.plot()
#plt.show()


weekly_decompose_df = weekly_decompose.seasonal.to_frame('seasonal')
weekly_decompose_df['weekday'] = weekly_decompose_df.index.day_name()
weekly_summary = weekly_decompose_df.groupby('weekday')['seasonal'].mean()
print(weekly_summary)


monthly_decompose = seasonal_decompose(df['y'], model='additive', period=30)
#weekly_decompose.plot()
#plt.show()


monthly_decompose_df = monthly_decompose.seasonal.to_frame('seasonal')
monthly_decompose_df['day_of_month'] = monthly_decompose_df.index.day
monthly_summary = monthly_decompose_df.groupby('day_of_month')['seasonal'].mean()
print(monthly_summary)

#Add seasonal components
#Add plot_components



0
weekday
Friday       -5.357943
Monday        9.626143
Saturday      2.863308
Sunday      -15.070036
Thursday      4.437425
Tuesday       1.666229
Wednesday     1.834875
Name: seasonal, dtype: float64
day_of_month
1     5.475980
2     6.156577
3     7.291146
4     7.485842
5     7.153247
6     6.550616
7     5.747836
8     4.957040
9     4.949207
10    2.922090
11    1.372955
12    0.064537
13   -1.536805
14   -2.587647
15   -3.164535
16   -4.974232
17   -6.360247
18   -7.332069
19   -6.990535
20   -7.390990
21   -6.830323
22   -6.256391
23   -5.816793
24   -4.953474
25   -3.616230
26   -1.878762
27   -0.436591
28    1.038622
29    2.480673
30    3.450130
31    4.170650
Name: seasonal, dtype: float64


In [ ]:
#Logistic Regression with Fourier Conversions

m = Prophet(daily_seasonality=False)
m.add_seasonality(name='weekly', period=7, fourier_order=3)
m.add_seasonality(name='monthly', period=30, fourier_order=5)
m.add_seasonality(name='quarterly', period=90, fourier_order=5)
m.add_seasonality(name='annual', period=365, fourier_order=10)
m.add_country_holidays(country_name='PAK')

m.fit(df)

df_future = m.make_future_dataframe(periods=30)
forecast = m.predict(df_future)
m.plot_components(forecast)
plt.show()


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

df_cv = cross_validation(m, initial = '540 days', period = '30 days', horizon = '30 days')

df_p = performance_metrics(df_cv)
df_p.to_excel('df_p.xlsx')
df_cv.to_excel('df_cv.xlsx')

In [ ]:
from google.colab import drive

forecast.to_excel('/content/forecast.xlsx', index=False)

In [ ]:
!git clone https://github.com/Sannan27/glowing-adventure.git

%cd glowing-adventure

!git config user.name "abc"
!git config user.email "abc"

!git add .
!git commit -m"Recharge Forecast v 1.0"
!git push

In [ ]:
%cd /content
!rm -rf glowing-adventure

/content
